In [161]:
import math
import numpy as np
import pandas as pd

In [162]:
# Training and test data files
train_file = '../input/train.csv'
test_file = '../input/test.csv'
model_file = '../output/titanic.model.json'
model_weights_file = '../output/titanic.model.best.hdf5'
pred_file = '../output/gender_submission.csv'

In [163]:
# Load training data
df_train = pd.read_csv(train_file)
print(df_train.shape)
df_train.head()

(891, 12)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [164]:
# Extract survived data as predictions: 0 = Died, 1 = Survived
from keras.utils.np_utils import to_categorical
y_train = to_categorical(df_train["Survived"], 2)
df_train.pop('Survived')
print(y_train.shape)
print(y_train[0:5])

(891, 2)
[[ 1.  0.]
 [ 0.  1.]
 [ 0.  1.]
 [ 0.  1.]
 [ 1.  0.]]


In [165]:
# Prepare the rest of the data for training
from sklearn.preprocessing import MinMaxScaler

max_name_len = df_train.Name.map(len).max()
max_ticket_len = df_train.Ticket.map(len).max()

title_list=['Mrs', 'Mr', 'Master', 'Miss', 'Major', 'Rev', 'Dr', 'Ms', 'Mlle',
            'Col', 'Capt', 'Mme', 'Countess', 'Don', 'Jonkheer']

import string
def substrings_in_string(big_string, substrings):
    for substring in substrings:
        if substring in big_string:
            return substring
    return np.nan

def prep_data(frame):
    # Creating new family_size and fare per person columns 
    frame['Family_Size'] = frame['SibSp'] + frame['Parch'] + 1

    # Convert Sex and Embarked to number
    frame['Sex'] = pd.Categorical(frame['Sex']).codes
    frame['Embarked'] = pd.Categorical(frame['Embarked']).codes
    
    # Extract title from name
    frame['Title'] = frame['Name'].map(lambda x: substrings_in_string(x, title_list))
    frame['Title'] = pd.Categorical(frame['Title']).codes

    # Convert Name into characters
    frame['Name_Length'] = frame.apply(lambda row: len(row['Name']), axis=1)
    for i in range(0, max_name_len):
        col_name = 'Name' + str(i)
        frame[col_name] = frame['Name'].str[i]
        frame[col_name] = frame.apply(lambda row: 0 if isinstance(row[col_name], float) and math.isnan(row[col_name]) else ord(row[col_name]), axis=1)
    frame.pop('Name')    
    
    # Convert Ticket into characters
    frame['Ticket_Length'] = frame.apply(lambda row: len(row['Ticket']), axis=1)
    for i in range(0, max_ticket_len):
        col_name = 'Ticket' + str(i)
        frame[col_name] = frame['Ticket'].str[i]
        frame[col_name] = frame.apply(lambda row: 0 if isinstance(row[col_name], float) and math.isnan(row[col_name]) else ord(row[col_name]), axis=1)
    frame.pop('Ticket')    
    
    # Convert Cabin column to whether in cabin
    frame['In_Cabin'] = frame.apply(lambda row: 1 if row['Cabin'] != 0 else 0, axis=1)
    frame.pop('Cabin')
    
    # Change any remaining N/As to 0
    frame = frame.fillna(0)

    print("Before scaling: ")
    print(frame.head())
    
    # Scale everything except PassengerId
    min_max_scaler = MinMaxScaler()
    col_list = frame.columns.tolist()
    col_list.remove('PassengerId')
    frame = frame[col_list]
    np_scaled = min_max_scaler.fit_transform(frame)
    frame = pd.DataFrame(np_scaled)
    
    print("After scaling: ")
    print(frame.head())

    return frame


In [166]:
# Prepare the rest of the data for training
df_train = prep_data(df_train)
X_train = np.array(df_train)[:,:]
X_train = X_train.astype('float32')
print(X_train.shape)
print(X_train[0])

Before scaling: 
   PassengerId  Pclass  Sex   Age  SibSp  Parch     Fare  Embarked  \
0            1       3    1  22.0      1      0   7.2500         2   
1            2       1    0  38.0      1      0  71.2833         0   
2            3       3    0  26.0      0      0   7.9250         2   
3            4       1    0  35.0      1      0  53.1000         2   
4            5       3    1  35.0      0      0   8.0500         2   

   Family_Size  Title    ...     Ticket9  Ticket10  Ticket11  Ticket12  \
0            2     11    ...           0         0         0         0   
1            2     12    ...           0         0         0         0   
2            1      8    ...          51        49        48        49   
3            2     12    ...           0         0         0         0   
4            1     11    ...           0         0         0         0   

   Ticket13  Ticket14  Ticket15  Ticket16  Ticket17  In_Cabin  
0         0         0         0         0         0  

In [186]:
# Build a training network

from keras.models import Sequential
from keras.layers import Dense, Dropout, RepeatVector, Flatten, Activation
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.optimizers import SGD
from keras.layers.advanced_activations import LeakyReLU

def create_model(nodes = 512, layers = 2, optimizer='sgd'):
    model = Sequential()
    model.add(Dense(nodes, activation='relu', input_shape=(X_train.shape[1],)))
    for i in range(1, layers):
        model.add(Dropout(0.2))
        model.add(Dense(nodes, activation='relu'))
    model.add(Dense(2, activation='sigmoid'))
    model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    return model

In [187]:
# Save the model
#model_json = model.to_json()
#with open(model_file, 'w') as json_file:
#    json_file.write(model_json)

In [194]:
# Grid search
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

model = KerasClassifier(build_fn=create_model, epochs=100, batch_size=10, verbose=0)

# Define the grid search parameters
nodes = range(32, X_train.shape[0], 32)
layers = range(1, 3)
optimizers = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
param_grid = dict(nodes=nodes, layers=layers, optimizer=optimizers)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, verbose=2)
grid_result = grid.fit(X_train, y_train)

Fitting 3 folds for each of 378 candidates, totalling 1134 fits
[CV] layers=1, nodes=32, optimizer=SGD ...............................
[CV] layers=1, nodes=32, optimizer=SGD ...............................
[CV] layers=1, nodes=32, optimizer=SGD ...............................
[CV] layers=1, nodes=32, optimizer=RMSprop ...........................


KeyboardInterrupt: 

In [ ]:
# Get the best parameters
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

In [178]:
# Train the model
#checkpointer = ModelCheckpoint(filepath=model_weights_file, verbose=1, save_best_only=True)
#stopper = EarlyStopping(monitor='val_loss', min_delta=1e-4, patience=25, verbose=1, mode='auto')
#hist = model.fit(X_train, y_train, epochs=1000, batch_size=20, validation_split=0.25, 
#                 callbacks=[checkpointer, stopper], 
#                 verbose=1, shuffle=True)

Train on 668 samples, validate on 223 samples
Epoch 1/1000
668/668 [==============================] - 1s - loss: 0.6846 - acc: 0.5793 - val_loss: 0.6655 - val_acc: 0.6368
Epoch 2/1000
668/668 [==============================] - 0s - loss: 0.6631 - acc: 0.6168 - val_loss: 0.6465 - val_acc: 0.6323
Epoch 3/1000
668/668 [==============================] - 0s - loss: 0.6508 - acc: 0.6228 - val_loss: 0.6318 - val_acc: 0.6323
Epoch 4/1000
668/668 [==============================] - 0s - loss: 0.6387 - acc: 0.6257 - val_loss: 0.6173 - val_acc: 0.6368
Epoch 5/1000
668/668 [==============================] - 0s - loss: 0.6258 - acc: 0.6377 - val_loss: 0.6058 - val_acc: 0.7085
Epoch 6/1000
668/668 [==============================] - 0s - loss: 0.6163 - acc: 0.6512 - val_loss: 0.5933 - val_acc: 0.7265
Epoch 7/1000
668/668 [==============================] - 0s - loss: 0.6022 - acc: 0.6662 - val_loss: 0.5817 - val_acc: 0.7489
Epoch 8/1000
668/668 [==============================] - 0s - loss: 0.5943 - acc

668/668 [==============================] - 0s - loss: 0.4694 - acc: 0.7799 - val_loss: 0.4488 - val_acc: 0.7937
Epoch 28/1000
668/668 [==============================] - 0s - loss: 0.4679 - acc: 0.8024 - val_loss: 0.4464 - val_acc: 0.7982
Epoch 29/1000
668/668 [==============================] - 0s - loss: 0.4790 - acc: 0.7814 - val_loss: 0.4438 - val_acc: 0.7982
Epoch 30/1000
668/668 [==============================] - 0s - loss: 0.4615 - acc: 0.7979 - val_loss: 0.4420 - val_acc: 0.8027
Epoch 31/1000
668/668 [==============================] - 0s - loss: 0.4594 - acc: 0.7814 - val_loss: 0.4502 - val_acc: 0.7982
Epoch 32/1000
668/668 [==============================] - 0s - loss: 0.4621 - acc: 0.7994 - val_loss: 0.4386 - val_acc: 0.8072
Epoch 33/1000
668/668 [==============================] - 0s - loss: 0.4692 - acc: 0.7904 - val_loss: 0.4382 - val_acc: 0.8072
Epoch 34/1000
668/668 [==============================] - 0s - loss: 0.4526 - acc: 0.7934 - val_loss: 0.4350 - val_acc: 0.8027
Epoch 

668/668 [==============================] - 0s - loss: 0.4318 - acc: 0.8159 - val_loss: 0.4211 - val_acc: 0.8117
Epoch 57/1000
668/668 [==============================] - 0s - loss: 0.4183 - acc: 0.8129 - val_loss: 0.4199 - val_acc: 0.8117
Epoch 58/1000
668/668 [==============================] - 0s - loss: 0.4170 - acc: 0.8234 - val_loss: 0.4291 - val_acc: 0.8027
Epoch 59/1000
668/668 [==============================] - 0s - loss: 0.4236 - acc: 0.8174 - val_loss: 0.4194 - val_acc: 0.8117
Epoch 60/1000
668/668 [==============================] - 0s - loss: 0.4181 - acc: 0.8189 - val_loss: 0.4198 - val_acc: 0.8161
Epoch 61/1000
668/668 [==============================] - 0s - loss: 0.4346 - acc: 0.8159 - val_loss: 0.4243 - val_acc: 0.8072
Epoch 62/1000
668/668 [==============================] - 0s - loss: 0.4260 - acc: 0.8099 - val_loss: 0.4205 - val_acc: 0.8161
Epoch 63/1000
668/668 [==============================] - 0s - loss: 0.4105 - acc: 0.8114 - val_loss: 0.4182 - val_acc: 0.8072
Epoch 

668/668 [==============================] - 0s - loss: 0.4024 - acc: 0.8383 - val_loss: 0.4146 - val_acc: 0.8206
Epoch 87/1000
668/668 [==============================] - 0s - loss: 0.3955 - acc: 0.8249 - val_loss: 0.4131 - val_acc: 0.8206
Epoch 88/1000
668/668 [==============================] - 0s - loss: 0.3839 - acc: 0.8323 - val_loss: 0.4126 - val_acc: 0.8161
Epoch 89/1000
668/668 [==============================] - 0s - loss: 0.3903 - acc: 0.8413 - val_loss: 0.4139 - val_acc: 0.8251
Epoch 90/1000
668/668 [==============================] - 0s - loss: 0.3972 - acc: 0.8263 - val_loss: 0.4376 - val_acc: 0.8027
Epoch 91/1000
668/668 [==============================] - 0s - loss: 0.4008 - acc: 0.8234 - val_loss: 0.4197 - val_acc: 0.8206
Epoch 92/1000
668/668 [==============================] - 0s - loss: 0.3907 - acc: 0.8443 - val_loss: 0.4167 - val_acc: 0.8251
Epoch 93/1000
668/668 [==============================] - 0s - loss: 0.3872 - acc: 0.8503 - val_loss: 0.4136 - val_acc: 0.8117
Epoch 

668/668 [==============================] - 0s - loss: 0.3575 - acc: 0.8413 - val_loss: 0.4278 - val_acc: 0.8161
Epoch 118/1000
668/668 [==============================] - 0s - loss: 0.3622 - acc: 0.8443 - val_loss: 0.4221 - val_acc: 0.8161
Epoch 119/1000
668/668 [==============================] - 0s - loss: 0.3641 - acc: 0.8338 - val_loss: 0.4235 - val_acc: 0.8161
Epoch 120/1000
668/668 [==============================] - 0s - loss: 0.3616 - acc: 0.8593 - val_loss: 0.4107 - val_acc: 0.8206
Epoch 121/1000
668/668 [==============================] - 0s - loss: 0.3659 - acc: 0.8503 - val_loss: 0.4094 - val_acc: 0.8251
Epoch 122/1000
668/668 [==============================] - 0s - loss: 0.3559 - acc: 0.8608 - val_loss: 0.4115 - val_acc: 0.8206
Epoch 123/1000
668/668 [==============================] - 0s - loss: 0.3593 - acc: 0.8488 - val_loss: 0.4137 - val_acc: 0.8161
Epoch 124/1000
668/668 [==============================] - 0s - loss: 0.3496 - acc: 0.8623 - val_loss: 0.4154 - val_acc: 0.8161

In [179]:
# Load the weights that yielded the best validation accuracy
model.load_weights(model_weights_file)

# Evaluate the model on the training set
score = model.evaluate(X_train, y_train)
print("\nTraining Accuracy:", score[1])

672/891 [=====================>........] - ETA: 0s
Training Accuracy: 0.867564535101


In [180]:
# Load test data
df_test_raw = pd.read_csv(test_file)
print(df_test_raw.shape)
df_test_raw.head()

(418, 11)


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [181]:
# Prepare the data for testing
df_test = prep_data(df_test_raw)
X_test = np.array(df_test)[:,:]
X_test = X_test.astype('float32')
print(X_test.shape)
print(X_test[0])

Before scaling: 
   PassengerId  Pclass  Sex   Age  SibSp  Parch     Fare  Embarked  \
0          892       3    1  34.5      0      0   7.8292         1   
1          893       3    0  47.0      1      0   7.0000         2   
2          894       2    1  62.0      0      0   9.6875         1   
3          895       3    1  27.0      0      0   8.6625         2   
4          896       3    0  22.0      1      1  12.2875         2   

   Family_Size  Title    ...     Ticket9  Ticket10  Ticket11  Ticket12  \
0            1      5    ...           0         0         0         0   
1            2      6    ...           0         0         0         0   
2            1      5    ...           0         0         0         0   
3            1      5    ...           0         0         0         0   
4            3      6    ...           0         0         0         0   

   Ticket13  Ticket14  Ticket15  Ticket16  Ticket17  In_Cabin  
0         0         0         0         0         0  

In [182]:
# Predict for test data
y_test = model.predict(X_test)
print(y_test[0])

[ 0.28025097  0.05451864]


In [183]:
# Save predictions
with open(pred_file, 'w') as f:
    f.write('PassengerId,Survived\n')
    for index, y_hat in enumerate(y_test):
        prediction = np.argmax(y_hat)
        f.write(str(int(df_test_raw.iloc[index]['PassengerId'])) + ',' + str(prediction)+'\n')
    f.close()